In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from pathlib import Path

In [2]:
import sys

sys.path.append(str(Path("..").resolve()))

from src.output_creator import OutputCreator

In [3]:
ROOT_DIR = Path.cwd().parent 

ruta_datos = ROOT_DIR / "data" / "spotify_songs.csv"

ruta_graficos = ROOT_DIR / "outputs" / "graficos"

ruta_tablas = ROOT_DIR / "outputs" / "tablas"

df = pd.read_csv(ruta_datos)

In [4]:
saver = OutputCreator(ruta_tablas, ruta_graficos)

In [5]:
variables_irrelevantes = ["track_id", "track_album_id", "playlist_id"]
df = df.drop(columns=variables_irrelevantes)

In [6]:
df_size = len(df)
df_size

32833

# Fase 1: Comprensión del negocio

## Contexto

---

## Objetivo de Negocio

---

## Objetivo del Proyecto de Datos

---

## KPIs (Indicadores Clave)


---

# Fase 2: Comprensión De Los Datos


## 2.1 Diccionario de Datos:

| variable | class | description |
| :--- | :--- | :--- |
| `track_name` | character (categorica nominal) | Nombre de la canción |
| `track_artist` | character (categorica nominal) | Artista |
| `track_album_name` | character (categorica nominal) | Nombre del Album |
| `playlist_name` | character (categorica nominal) | Nombre de la playlist |
| `playlist_genre` | character (categorica nominal) | Genero de la playlist |
| `playlist_subgenre` | character (categorica nominal) | Subgenero de la playlist |
| `mode` | double (categorica nominal) | El modo indica la modalidad (mayor o menor) de una pista, el tipo de escala de la que se deriva su contenido melódico. El mayor se representa con 1 y el menor con 0. |
| `track_popularity` | double (cuantitativa continua) | Popularidad de la canción (0-100) minetras mas alta sea mejor |
| `danceability` | double (cuantitativa continua) | La **danceability** o **bailabilidad** describe la idoneidad de una pista para bailar según una combinación de elementos musicales, como el tempo, la estabilidad rítmica, la intensidad del compás y la regularidad general. Un valor de 0.0 es el menos bailable y 1.0 el más bailable. |
| `energy` | double (cuantitativa continua) | La energía es una medida de 0.0 a 1.0 y representa una medida perceptual de intensidad y actividad. Normalmente, las pistas energéticas se perciben rápidas, fuertes y ruidosas. Por ejemplo, el death metal tiene mucha energía, mientras que un preludio de Bach tiene una puntuación baja en la escala. Las características perceptuales que contribuyen a este atributo incluyen el rango dinámico, la sonoridad percibida, el timbre, la velocidad de inicio y la entropía general. |
| `key` | double (preparada numericamente), en el negocio es character (categorica nominal) | La tonalidad general estimada de la pista. Los números enteros se asignan a tonos utilizando la notación estándar de clases de tono. Por ejemplo, 0 = C, 1 = C♯/D♭, 2 = D, y así sucesivamente. Si no se detecta ninguna tonalidad, el valor es -1. |
| `tempo` | double (cuantitativa continua) | El tempo general estimado de una pista en pulsaciones por minuto (BPM). En terminología musical, el tempo es la velocidad o el ritmo de una pieza determinada y se deriva directamente de la duración media de los compases. |
| `duration_ms` | double (cuantitativa continua) | Duración de la canción en milisegundos |
| `loudness` | double (cuantitativa continua) | La sonoridad general de una pista en decibelios (dB). Los valores de sonoridad se promedian en toda la pista y son útiles para comparar la sonoridad relativa de las pistas. La sonoridad es el volumen percibido por el oído humano a lo largo del tiempo, independientemente de la potencia real de los altavoces. Los valores suelen oscilar entre -60 y 0 db. |
| `speechiness` | double (cuantitativa continua) | La "speechiness" detecta la presencia de palabras habladas en una pista. Cuanto más exclusivamente parecida al habla sea la grabación (por ejemplo, un programa de entrevistas, un audiolibro, poesía), más cerca de 1.0 estará el valor del atributo. Los valores superiores a 0.66 describen pistas que probablemente estén compuestas en su totalidad por palabras habladas. Los valores entre 0.33 y 0.66 describen pistas que pueden contener tanto música como habla, ya sea en secciones o en capas, incluyendo casos como la música rap. Los valores inferiores a 0.33 representan probablemente música y otras pistas que no son de tipo habla. |
| `acousticness` | double (cuantitativa continua) | Una medida de confianza de 0.0 a 1.0 sobre si la pista es acústica. 1.0 representa una confianza alta de que la pista es acústica. |
| `instrumentalness` | double (cuantitativa continua) | Predice si una pista no contiene voces. Los sonidos "Ooh" y "aah" se tratan como instrumentales en este contexto. Las pistas de rap o de palabras habladas son claramente "vocales". Cuanto más cercano a 1.0 sea el valor de instrumentalidad, mayor será la probabilidad de que la pista no contenga contenido vocal. Los valores superiores a 0.5 están destinados a representar pistas instrumentales, pero la confianza es mayor a medida que el valor se acerca a 1.0. |
| `valence` | double (cuantitativa continua) | Una medida de 0.0 a 1.0 que describe la positividad musical transmitida por una pista. Las pistas con una valencia alta suenan más positivas (por ejemplo, felices, alegres, eufóricas), mientras que las pistas con una valencia baja suenan más negativas (por ejemplo, tristes, deprimidas, enfadadas). |

<p align="left">
  <small><b>Tabla 1:</b> Diccionario de datos adaptado de <a href="https://www.kaggle.com/datasets/joebeachcapital/30000-spotify-songs">30,000 Spotify Songs</a> por <a href="https://www.kaggle.com/joebeachcapital">Joakim Arvidsson</a>. Clasificación estadística añadida por el equipo de trabajo.</small>
</p>

---
## 2.2 Relevancia
Todas aquellas variables relevantes para el objetivo ya estan incluidas en el diccionario de datos, aquellas que no son utiles para el objetivo se omitieron en la tabla presentada. Dichas variables son utiles en el contexto, pues describen tanto caracteristicas propias de las pistas de audio como tambien sus metadatos (ej: artista), este conjunto de datos sera util para generar varias asociaciónes, correlaciones o establecer relaciones entre variables categoricas y numericas, de esta manera permitiendo al equipo de investigación extraer insights estratégicos para el negocio.




## 2.3 Diagnostico inicial de calidad

### 2.3.1 Identificación de nulos y su proporción

In [7]:
columnas_con_nulos = df.isna().sum().to_frame(name='Total_Nulos')
columnas_con_nulos

,Total_Nulos
track_name,5
track_artist,5
track_popularity,0
track_album_name,5
track_album_release_date,0
playlist_name,0
playlist_genre,0
playlist_subgenre,0
danceability,0
energy,0


Solo 3 variables tienen valores nulos (`track_name`, `track_artist` y `track_album_name`), ademas es una proporcion realmente baja de nulos, a lo sumo hay una proporcion de $\frac{15}{32833}$ registros con nulos, lo cual correstponde a tan solo el **$ 0.0456\%$** de la muestra

In [8]:
saver.guardar_csv(columnas_con_nulos, "columnas_con_nulos.csv")

El archivo ya existe en: d:\Externado\tercer semestre\Data Mining\Proyecto_Data_Mining_UEC\outputs\tablas\columnas_con_nulos.csv
Operación cancelada para evitar duplicados.


In [9]:
registros_nulos = df[df.isnull().any(axis=1)]
registros_nulos

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
8151,NaN,NaN,0,NaN,2012-01-05,HIP&HOP,rap,southern hip hop,0.714,0.821,6,-7.635,1,0.1760,0.0410,0.00000,0.1160,0.649,95.999,282707
9282,NaN,NaN,0,NaN,2017-12-01,GANGSTA Rap,rap,gangster rap,0.678,0.659,11,-5.364,0,0.3190,0.0534,0.00000,0.5530,0.191,146.153,202235
9283,NaN,NaN,0,NaN,2017-12-01,GANGSTA Rap,rap,gangster rap,0.465,0.820,10,-5.907,0,0.3070,0.0963,0.00000,0.0888,0.505,86.839,206465
19568,NaN,NaN,0,NaN,2012-01-05,Reggaeton viejito🔥,latin,reggaeton,0.675,0.919,11,-6.075,0,0.0366,0.0606,0.00653,0.1030,0.726,97.017,252773
19811,NaN,NaN,0,NaN,2012-01-05,latin hip hop,latin,latin hip hop,0.714,0.821,6,-7.635,1,0.1760,0.0410,0.00000,0.1160,0.649,95.999,282707


In [10]:
saver.guardar_csv(registros_nulos, "registros_nulos.csv")

El archivo ya existe en: d:\Externado\tercer semestre\Data Mining\Proyecto_Data_Mining_UEC\outputs\tablas\registros_nulos.csv
Operación cancelada para evitar duplicados.


Los 5 nulos que se observaron en cada una de las tres variables mencionadas anteriormente pertenecen a lo mismos registros, por lo que solo el $0.01522 \%$ de los registros de la base de datos contiene nulos, una proporción despreciable. La mejor opción a futuro será la eliminación.

### 2.3.2 Revisión de formatos incosistentes

In [11]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_name                32828 non-null  str    
 1   track_artist              32828 non-null  str    
 2   track_popularity          32833 non-null  int64  
 3   track_album_name          32828 non-null  str    
 4   track_album_release_date  32833 non-null  str    
 5   playlist_name             32833 non-null  str    
 6   playlist_genre            32833 non-null  str    
 7   playlist_subgenre         32833 non-null  str    
 8   danceability              32833 non-null  float64
 9   energy                    32833 non-null  float64
 10  key                       32833 non-null  int64  
 11  loudness                  32833 non-null  float64
 12  mode                      32833 non-null  int64  
 13  speechiness               32833 non-null  float64
 14  acousticness     

todas las variables tienen el Dtype adecuado

### 2.4 Detección de registros duplicados.

En este caso se consideran duplicados registros de canciones que repiten nombre y autor, y la unica diferencia que presentan es la playlist de la cual fueron tomadas desde la API, es decir son la misma cancion, solo que tomada de una playlist diferente

In [28]:
duplicados = df.duplicated(subset=['track_name', 'track_artist'], keep='first')

df_duplicados = df[duplicados]
ejemplo_de_canciones_duplicadas = df_duplicados[df_duplicados['track_name'] == 'Callaita']
ejemplo_de_canciones_duplicadas

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
11135,Callaita,Bad Bunny,90,Callaita,2019-05-31,Trap 2020 🔥,rap,trap,0.61,0.624,2,-4.773,1,0.309,0.6,0.000002,0.243,0.244,176.169,250534
17872,Callaita,Bad Bunny,90,Callaita,2019-05-31,LATIN POP 2020 🔥 Pop latino actual,latin,latin pop,0.61,0.624,2,-4.773,1,0.309,0.6,0.000002,0.243,0.244,176.169,250534
18816,Callaita,Bad Bunny,90,Callaita,2019-05-31,Mansión Reggaetón,latin,reggaeton,0.61,0.624,2,-4.773,1,0.309,0.6,0.000002,0.243,0.244,176.169,250534
20282,Callaita,Bad Bunny,90,Callaita,2019-05-31,Urbano: Latino Vibes,latin,latin hip hop,0.61,0.624,2,-4.773,1,0.309,0.6,0.000002,0.243,0.244,176.169,250534
21893,Callaita,Bad Bunny,90,Callaita,2019-05-31,Chixtape 5 - Tory Lanez,r&b,urban contemporary,0.61,0.624,2,-4.773,1,0.309,0.6,0.000002,0.243,0.244,176.169,250534


In [30]:
saver.guardar_csv(ejemplo_de_canciones_duplicadas, 'ejemplo_de_canciones_duplicadas.csv')

Archivo guardado exitosamente en: d:\Externado\tercer semestre\Data Mining\Proyecto_Data_Mining_UEC\outputs\tablas\ejemplo_de_canciones_duplicadas.csv


Vea que la cancion *Callaita* se repite numerosas veces en los registros, sin embargo la unica diferencia entre estas ocurrencias radica en su playlist de origen. A futuro se sugioere dejar solo un registro y agregar la variable ocurrencias en playlists para no perder esta información, o alternativamente en una variable "playlists" agregar una cadena que contenga todas las playlist que contienen la cancion, logrando de esta manera un solo registro por pista.

In [20]:
len(df_duplicados)

6603

En total hay 6603 canciones duplicadas en la base de datos (solo se tomaron en cuenta las ocurrencias sin incluir la primera fila, esa no se toma como duplicado)

## 3 EDA

## 3.1 Analisis Univariado

### 3.1.1 Variables categoricas.

#### track_name                

In [12]:
len(df['track_name'].unique())

23450

In [13]:
df['track_name'].mode()

0    Poison
Name: track_name, dtype: str

hay 23450 valores unicos, no vale la pena graficar esta varibale, el titulo mas comun en canciones es Poison (probablemente alguna cancion con ese nombre se repite varias veces en la base de datos)

#### track_artist

In [31]:
len(df['track_artist'].unique())

10693

In [32]:
df['track_artist'].mode()

0    Martin Garrix
Name: track_artist, dtype: str

De los 10693 artistas distintos que contiene la base de datos el que mas suele repetirse es Martin Garrix, probablemente sus canciones tengan alta presencia en playlists

#### track_album_name

In [33]:
len(df['track_album_name'].unique())

19744

In [34]:
df['track_album_name'].mode()

0    Greatest Hits
Name: track_album_name, dtype: str

de los 19744 albumnes distintos el que mas se repite es una compilacion de *Greatest Hits*

#### playlist_genre

In [35]:
len(df['playlist_genre'].unique())

6

In [38]:
print(f"generos musicales considerados: {list(df['playlist_genre'].unique())}")

generos musicales considerados: ['pop', 'rap', 'rock', 'latin', 'r&b', 'edm']
